# Using Prompts Inline
We need to frist complete the same basic steps we did in the other modules.


## 1 - Load the Kernel

In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Initialize the kernel
kernel = Kernel()


## 2 - Load Azure OpenAI Endpoint details.

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Set path to the .env file in the notebooks directory
notebooks_dir = Path(os.path.abspath('')).parent.parent
env_path = notebooks_dir / '.env'

# Load environment variables from the notebooks/.env file
load_dotenv(dotenv_path=env_path)

# Set up environment variables
# You can also create a .env file with the following variables
# AZURE_OPENAI_API_KEY=your-api-key
# AZURE_OPENAI_ENDPOINT=your-endpoint

# Check if environment variables are set
def check_env_vars():
    required_vars = ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT"]
    missing_vars = [var for var in required_vars if not os.environ.get(var)]
    
    if missing_vars:
        print(f"Warning: Missing environment variables: {', '.join(missing_vars)}")
        print(f"Please create a .env file at {env_path} with these variables.")
        print("Some examples in this notebook may not work without these variables.")
    else:
        print("All required environment variables are set!")
        
        # Display part of the API key and endpoint for verification
        api_key = os.environ.get("AZURE_OPENAI_API_KEY", "")
        endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT", "")
        
        # Show only the first 5 and last 4 characters of the API key
        if api_key:
            masked_key = api_key[:5] + "*" * (len(api_key) - 9) + api_key[-4:] if len(api_key) > 9 else "Not found"
            print(f"API Key: {masked_key} (masked for security)")
            
        # Show the endpoint domain
        if endpoint:
            print(f"Endpoint: {endpoint}")

# Uncomment to check environment variables
# check_env_vars()

## 3- Add Chat Completion Service

In [3]:
import os

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Get values from environment variables
deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")

# Add Azure OpenAI chat completion service
chat_service = AzureChatCompletion(
    deployment_name=deployment_name,
    endpoint=endpoint,
    api_key=api_key
    # model_id="gpt-4"  # Optional if deployment name matches model
)

# kernel.add_service(chat_service)

print(f"Added chat completion service using deployment: {deployment_name}")

Added chat completion service using deployment: gpt-4o


## 4 - Construct the Kernel, ChatHistory and get instance of the ChatCompletion Service

In [4]:
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory

# Get the chat service from kernel
kernel.add_service(chat_service) 

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
        
history = ChatHistory(system_message="You are a helpful assistant.")
history.add_user_message("Why is the sky blue?")

## 5 - Let's create a prompt Template and use it

In [ ]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.functions import KernelArguments

promptPiglatin = """++++
    Convert the follow to Pig Latin: 
    {{$input}}
    ++++

    Pig Latin Translation: 
"""

# Define the prompt template config
prompt_template_piglatin = PromptTemplateConfig(
    template=promptPiglatin,
    name="PigLatin",
    template_format="semantic-kernel",
    variables=[
        InputVariable(name="input", description="The user input to convert to Pig Latin")
    ],
    execution_settings=execution_settings,
)

kernel.ask

func = kernel.add_function(
    function_name="pigLatinFunc",
    plugin_name="pigLatinPlugin",
    prompt_template_config=prompt_template_piglatin,
)
input_text = """Why is the sky blue?"""



# Invoke the function
result = await kernel.invoke(func, input=input_text)

# Print the result
print("AI Response:", result)

AI Response: Sure! Here's the Pig Latin translation of "Why is the sky blue?":

**Pig Latin Translation:**  
"Ywhay isyay ethay kysay ueblay?"
